# mscp python binding example

In [60]:
import glob
import time
import os

import mscp

In [53]:
this_dir = os.getcwd()

## Simple copy

In [54]:
# preparing files to be transferred
c_sources = glob.glob("../src/*.c")
c_sources

['../src/ssh.c',
 '../src/mscp.c',
 '../src/platform.c',
 '../src/pymscp.c',
 '../src/main.c',
 '../src/path.c',
 '../src/message.c',
 '../src/fileops.c']

In [55]:
# copy files using mscp
dst_dir = this_dir + "/simple-copy-dest"
m = mscp.mscp("localhost", mscp.LOCAL2REMOTE)
m.copy(c_sources, dst_dir)

In [56]:
os.listdir("simple-copy-dest")

['ssh.c',
 'mscp.c',
 'platform.c',
 'pymscp.c',
 'main.c',
 'path.c',
 'message.c',
 'fileops.c']

## Simple Copy with Python Rich ProgressBar

In [64]:
# make a 256MB file
src = "example-256MB-src.img"
with open(src, "wb") as f:
    f.seek(128 * 1024 * 1024 -1, 0)
    f.write(b'1')

In [69]:
# copy the 256MB file while ploting progress bar using python rich
dst = this_dir + "/example-256MB-dst.img"

kw = {"nr_threads": 1, "nr_ahead": 1} # slow mscp to watch the progress bar

m = mscp.mscp("localhost", mscp.LOCAL2REMOTE, **kw)
m.copy(src, dst, nonblock = True)

# m.stats() returns total bytes to be transferred, bytes transferred (done), and finished (bool).
total, done, finished = m.stats()
with Progress() as progress:

    task = progress.add_task(f"[green]Copying {src}", total = total)

    while not progress.finished:
        total, done, finished = m.stats()
        progress.update(task, completed = done)
        time.sleep(0.5)

m.join()
m.cleanup()

Output()